<a href="https://colab.research.google.com/github/leoscarlato/math-language-project/blob/main/Projeto_Linguagem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

NOME da linguagem: **{mat}**

POR: Alexandre Magno e Leonardo Scarlato


**{mat}** consiste em uma linguagem de programação interpretada, com a finalidade de facilitar a escrita em português de operações matemáticas. Nesta linguagem, há palavras reservadas que representam operações que proximam a dialética matemática oral com o código. Assim, proporciando um espaço mais amigavél para estudos de ambas ciencias.

O público alvo dessa linguagem são de jovens no estágio de desenvolvimento do estudo matemático e de programação.

**{mat}** lidará com números pertencentes ao conjunto dos Racionais(Q). Contudo, na linguagem, trabalha-se com a separação entre inteiros (int) e fracionarios (float).

In [2]:
!pip install rply
!pip install numpy

import numpy as np

In [6]:
# Analisador Lexico

from rply import LexerGenerator

lg = LexerGenerator()

lg.add("NUMBER", r"\d+")
lg.add("PLUS", r"\+")
lg.add("MINUS", r"\-")
lg.add("MUL", r"\*")
lg.add("DIV", r"/")
lg.add("ROOT", r"raiz_de")
lg.add("POW", r"elevado_a")
lg.add("FACT", r"fatorial")
lg.add("PI", r"pi")
lg.add("SIN", r"seno")
lg.add("COS", r"cosseno")
lg.add("TAN", r"tangente")

lg.add("INT", r"int")
lg.add("FLOAT", r"float")

lg.add('ID', r'[a-zA-z][a-zA-z0-9]*')
lg.add('COMP','==')
lg.add('COMP','!=')
lg.add('COMP','>=')
lg.add('COMP','>')
lg.add('COMP','<=')
lg.add('COMP','<')

lg.add('EQUALS', r"=")

lg.ignore('\s+')

lexer = lg.build()

In [ ]:
# Classes da Arvore Sintatica

# TODO Inserir classes ifElse, While

from rply.token import BaseBox

class Prog(BaseBox):
    def __init__(self,decls,stmts):
        self.decls = decls
        self.stmts = stmts
    
    def accept(self, visitor):
        visitor.visit_prog(self)

class Declatations(BaseBox):
    def __init__(self,decl,decls):
        self.decl = decl
        self.decls = decls

    def accept(self, visitor):
        visitor.visit_declarations(self)

class Declaration(BaseBox):
    def __init__(self,id,typee):
        self.id = id
        self.typee = typee
        
    def accept(self, visitor):
        visitor.visit_declaration(self)

class Statements(BaseBox):
    def __init__(self,stmt,stmts):
        self.stmt = stmt
        self.stmts = stmts

    def accept(self, visitor):
        visitor.visit_statements(self)

class Statement(BaseBox):
    def __init__(self,cmd):
        self.cmd = cmd

    def accept(self, visitor):
        visitor.visit_statement(self)

class Atrib(BaseBox):
    def __init__(self,id,expr):
        self.id = id
        self.expr = expr

    def accept(self, visitor):
        visitor.visit_atrib(self)

class Expr(BaseBox):
    def accept(self, visitor):
        method_name = 'visit_{}'.format(self.__class__.__name__.lower())
        visit = getattr(visitor, method_name)
        visit(self)

class Id(Expr):
    def __init__(self, value):
        self.value = value

class Number(Expr):
    def __init__(self, value):
        self.value = value

class BinaryOp(Expr):
    def __init__(self, left, right):
        self.left = left
        self.right = right

class Add(BinaryOp):
    pass

class Sub(BinaryOp):
    pass

class Mul(BinaryOp):
    pass

class Div(BinaryOp):
    pass

class Pow(BinaryOp):
    pass

class Root(BinaryOp):
    pass

class Fact(BinaryOp):
    pass

class Sin(BinaryOp):
    pass

class Cos(BinaryOp):
    pass

class Tan(BinaryOp):
    pass

In [2]:
# Analisador Sintatico

from rply import ParserGenerator

pg = ParserGenerator(
    # ['NUMBER', 'OPEN_PARENS', 'CLOSE_PARENS',
    #  'PLUS', 'MINUS', 'MUL', 'DIV', 'INT', 'FLOAT', 'ID',
    #  'EQUALS','COMP','IF','ELSE', 'WHILE', 'ROOT', 
    #  'POW', 'FACT', 'PI', 'SIN', 'COS', 'TAN'
    # ],

    # precedence = [
    #     ('left', ['PLUS', 'MINUS']),
    #     ('left', ['MUL', 'DIV']),
    #     ('left', ['ROOT', 'POW', 'SIN', 'COS', 'TAN']),
    # ]

    ['NUMBER', 'OPEN_PARENS', 'CLOSE_PARENS',
     'PLUS', 'MINUS', 'MUL', 'DIV', 'INT', 'FLOAT', 'ID',
     'EQUALS','COMP','IF','ELSE', 'WHILE', 'ROOT', 
     'POW', 'FACT', 'PI'
    ],

    precedence = [
        ('left', ['PLUS', 'MINUS']),
        ('left', ['MUL', 'DIV']),
        ('left', ['ROOT', 'POW']),
    ]
)

@pg.production('prog : declarations statements')
def prog(p):
    return Prog(p[0], p[1])

@pg.production('declarations : declaration')
def declarations(p):
    return Declatations(p[0], None)

@pg.production('declarations : declaration declarations')
def declarations(p):
    return Declatations(p[0], p[1])

@pg.production('declaration : INT ID')
def declaration(p):
    return Declaration(p[1].getstr(), "int")

@pg.production('declaration : FLOAT ID')
def declaration(p):
    return Declaration(p[1].getstr(), "float")

@pg.production('statements : openstatement')
def statements(p):
    return Statements(p[0], None)

@pg.production('statements : openstatement statements')
def statements(p):
    return Statements(p[0], p[1])

@pg.production('openstatement : atrib')
def statement_atrib(p):
    return Statement(p[0])

@pg.production('expression : ID')
def expression_id(p):
    return Id(p[0].getstr())

@pg.production('openstatement : IF OPEN_PARENS expression COMP expression CLOSE_PARENS openstatement')
def expression_ifelse1(p):
    return IfElse (p[2],p[3],p[4],p[6],None)

@pg.production('openstatement : IF OPEN_PARENS expression COMP expression CLOSE_PARENS closedstatement ELSE openstatement')
def expression_ifelse1(p):
    return IfElse (p[2],p[3],p[4],p[6],p[8])

@pg.production('closedstatement : IF OPEN_PARENS expression COMP expression CLOSE_PARENS closedstatement ELSE closedstatement')
def expression_ifelse1(p):
    return IfElse (p[2],p[3],p[4],p[6],p[8])

@pg.production('while : WHILE OPEN_PARENS expression COMP expression CLOSE_PARENS openstatement')
def expression_while(p):
    return While (p[2], p[3], p[4], p[6])

@pg.production('atrib : ID EQUALS expression SEMICOL')
def atrib(p):
    return Atrib(p[0].getstr(),p[2])

@pg.production('expression : NUMBER')
def expression_number(p):
    return Number(int(p[0].getstr()))

@pg.production('expression : PI')
def expression_number(p):
    return Number(np.pi)

@pg.production('expression : expression PLUS expression')
@pg.production('expression : expression MINUS expression')
@pg.production('expression : expression MUL expression')
@pg.production('expression : expression DIV expression')
@pg.production('expression : expression ROOT expression')
@pg.production('expression : expression POW expression')
@pg.production('expression : expression FACT')
# @pg.production('expression : SIN expression')
# @pg.production('expression : COS expression')
# @pg.production('expression : TAN expression')
def expression_binop(p):
    left = p[0]
    right = p[2]
    if p[1].gettokentype() == 'PLUS':
        return Add(left, right)
    elif p[1].gettokentype() == 'MINUS':
        return Sub(left, right)
    elif p[1].gettokentype() == 'MUL':
        return Mul(left, right)
    elif p[1].gettokentype() == 'DIV':
        return Div(left, right)
    elif p[1].gettokentype() == 'ROOT':
        return Root(left, right)
    elif p[1].gettokentype() == 'POW':
        return Pow(left, right)
    elif p[1].gettokentype() == 'FACT':
        return Fact(left, None)
    # elif p[1].gettokentype() == 'SIN':
    #     return Sin(left, None)
    # elif p[1].gettokentype() == 'COS':
    #     return Cos(left, None)
    # elif p[1].gettokentype() == 'TAN':
    #     return Tan(left, None)
    else:
        raise AssertionError('Oops, this should not be possible!')

parser = pg.build()

ModuleNotFoundError: No module named 'rply'